# 1. Load modules

In [1]:
%matplotlib widget 
#%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import matplotlib.ticker as ticker
from scipy import optimize
import os, sys

In [2]:
import xrdpy

ModuleNotFoundError: No module named 'xrdpy.src'

In [ ]:
import sys
sys.path.insert(0, '/local/MyGitHub/xrdpy')
import xrdpy
# %load_ext autoreload
# %autoreload 2

# 2. Set up the xrd-file path

In [ ]:
filepath = "/local/MyGitHub/xrdpy/tests/xrd_files/"
filename = "NT3334C_RSM_w2T-2T-105AlN-AlGaN_fast.xrdml"

In [ ]:
savefig = True
save_fig_path = '/local/MyGitHub/xrdpy/imgs/'

In [ ]:
gen_fns = xrdpy.general_fns(print_log='low')
xrd_ = xrdpy.xrd(print_log='low')
xrd_plt = xrdpy.plottings(save_figure_dir=save_fig_path, print_log='low')

# 3. Set default variables

#### Variable source: https://www.ioffe.ru/SVA/NSM/Semicond/

In [ ]:
alloy_type='ternary' 
structure_type='wz'

# Lattice parameters for AlN and GaN binaries
AlN_a, AlN_c = 3.112, 4.982
GaN_a, GaN_c = 3.189, 5.185    

# Cij parameters for AlN and GaN binaries
GaN_C13 = 106
GaN_C33 = 398
AlN_C13 = 108
AlN_C33 = 373

# Bowing parameters
bowing_a = 0.0
bowing_c = 0.0
bowing_C13 = 0.0
bowing_C33 = 0.0

# reciprocal axes are multiplited by this number to make enlarge axes
mul_fact = 10000

# a_lp_bin: [a_lattice_parameter bin_1, a_lattice_parameter bin_2, bowing]
# For ternary_wz we need followings: [a_lp_bin, c_lp_bin, c13_bin, c33_bin]
list_binary_parameters = [[AlN_a, GaN_a, bowing_a], [AlN_c, GaN_c, bowing_c],
                          [AlN_C13, GaN_C13, bowing_C13], [AlN_C33, GaN_C33, bowing_C33]]

# 4. Read xrd-file

## 4.1 Real space data

In [ ]:
lambda_wavelength, two_theta_values, omega_values, rsm_intesity  = \
    xrd_.xrd_read_data(xrd_file_name=os.path.join(filepath, filename))
# X-ray wave number
R = 1/lambda_wavelength

## 4.2 Reciprocal space conversion

In [ ]:
total_two_theta_in_row = np.shape(rsm_intesity)[1]
rec_space_x, rec_space_y = xrd_.Qxy(omega=omega_values, two_theta=two_theta_values, 
                                    total_two_theta_in_row=total_two_theta_in_row,
                                    R=R, mul_fact=mul_fact)

# 5. Plottings

## 5.1 Real space map

In [ ]:
print('Plotting real space map ...')
xlabel_text = r'2$\mathrm{\theta}$'
ylabel_text = r'$\omega$ / $2\theta$'
save_file_name = 'AlN_AlGaN_AlN_real_space.png' if savefig else None
_ = xrd_.xrd_plot(save_figure_dir=save_fig_path, save_file_name=save_file_name, 
                 mode="real_space_calc_omega_by_2theta", xaxis_label=xlabel_text,threshold_intensity=1e-5,
                 yaxis_label=ylabel_text, color_map='jet', color_scale='log', 
                 colorbar_label='Intensity (counts/s)', show_plot=True, dpi=75)
print('Done')

## 5.2 Reciprocal space map

In [ ]:
xlabel_text = f'Qx*{mul_fact} (rlu)'
ylabel_text = f'Qy*{mul_fact} (rlu)'
print('Plotting reciprocal space map ...')
save_file_name = 'AlN_AlGaN_AlN_reciprocal_space.png' if savefig else None
_ = xrd_plt.xrd_plot(save_file_name=save_file_name, x_values=rec_space_x, 
                     y_values=rec_space_y, z_values=rsm_intesity,
                     mode="reciprocal_space", xaxis_label=xlabel_text, threshold_intensity=0.1,
                     yaxis_label=ylabel_text, color_map='jet', color_scale='log',
                     colorbar_label='Intensity (counts/s)', dpi=75)
print('Done')

## 5.3 Perform post-calculations from maps

### 5.3.1 Estimate the compostion and strain relaxation of other peaks in reciprocal space map

In [ ]:
find_results_4_peak = [3730, 9905]

#### 5.3.1.1 Get or define the reference point in data
##### E.g. Here we use the highest maximum point as reference (AlN). This will be used as full strain line reference. You use the above figure to decide which point to use as reference.

In [ ]:
##============================================
# TBD: make a fitting function to get the maximum positions
max_intensity_pos = np.unravel_index(np.argmax(rsm_intesity, axis=None), rsm_intesity.shape)
x_coord_max = rec_space_x[max_intensity_pos]
y_coord_max = rec_space_y[max_intensity_pos]
reference_peak = [x_coord_max, y_coord_max]
print(f'Reference peak/point: {x_coord_max}, {y_coord_max}')
##============================================

#### 5.3.1.2 Calculate reference point theoretically

In [ ]:
# Calculating only for AlN as this is my reference point
rec_space_theor_ref_xy = xrd_.Qxy_theor(AlN_a, AlN_c, hkl='105') 

#### 5.3.1.3 Calculate shift required to map theoretically calculated reference point on original data

In [ ]:
# Calcuate shift amount
shift_xy_ = [x_coord_max-rec_space_theor_ref_xy[0], y_coord_max-rec_space_theor_ref_xy[1]]

#### 5.3.1.4 Calculate composition and relxation for the other peak

In [ ]:
## optimize_f_args = (peak_coordinate_for which_to find_results, list_binary_parameters, shift_xy_, mul_fact, alloy_type, structure_type, hkl)
optimize_f_args = (list_binary_parameters, shift_xy_, mul_fact, alloy_type, structure_type, '105')
sol, relaxation, terminal_points = xrd_.find_composition_strain_4_point(find_results_4_peak, reference_peak,
                                                                        optimize_f_args, comp_interval=[0, 1], 
                                                                        root_finding_method='brentq')

### 5.3.2 Calculate no-relaxation line (theoretically calulated + shifted to reference point)
#### Note: This sub-subsection is for visualization 

In [ ]:
comps_ = np.linspace(0.65,1,11) #np.arange(0, 1.0+1e-10, step=0.25) 
AlGaN_a, AlGaN_c, AlGaN_C13, AlGaN_C33, AlGaN_D = \
    gen_fns.alloy_parameters_from_binary(comps_, list_binary_parameters, 
                                         alloy_type=alloy_type, structure_type=structure_type)

#### 5.3.2.1 Completely relaxaed (no-strain) line

In [ ]:
rec_space_theor_shift_AlGaN = xrd_.Qxy_theor(AlGaN_a, AlGaN_c, shift=shift_xy_) # after shift

#### 5.3.2.2 No-relaxation (fully strained) line: considering elastic relaxation
###### Deformation potential(D) = -2C13/C33 ==> (x1, y2+D(x1-x2))

In [ ]:
rec_space_AlGaN_full_strain_y_prime = xrd_.get_full_strain_line(reference_peak, rec_space_theor_shift_AlGaN, AlGaN_D)[1]

#### 5.3.2.3 Plot all together

In [ ]:
#====================== Plot the map =================================
save_file_name = 'AlN_AlGaN_AlN_2reciprocal_space.png' if savefig else None
fig, ax, _ = xrd_plt.xrd_plot(save_file_name=None, x_values=rec_space_x, 
                              y_values=rec_space_y, z_values=rsm_intesity,threshold_intensity=1e-5,
                              mode="reciprocal_space", xaxis_label=xlabel_text,
                              yaxis_label=ylabel_text, color_map='jet', color_scale='log',
                              colorbar_label='Intensity (counts/s)')

#==================== add no-strain lines (red dash) ==================
ax.plot(rec_space_theor_shift_AlGaN[0], rec_space_theor_shift_AlGaN[1], color='r', ls='-.')

#==================== add reference lines (black dash) ================
ax.axhline(y=y_coord_max, ls='--', color='black')
ax.axvline(x=x_coord_max, ls='--', color='black')

#==================== add required peak (blue cross) ===================
ax.scatter(find_results_4_peak[0], find_results_4_peak[1], marker='o', color='c')

#========= add no_strain-strain connecting points (green dash) =========
for JJ in range(len(comps_)):
    ax.plot([rec_space_theor_shift_AlGaN[0][JJ], x_coord_max],
            [rec_space_theor_shift_AlGaN[1][JJ], rec_space_AlGaN_full_strain_y_prime[JJ]], 
            color='g', ls='-.', marker='.')

#========= add no_strain-strain connecting points (blue dash) =========
ax.plot(terminal_points[:,0], terminal_points[:,1], color='b', ls='-', marker='x')

if savefig:
    xrd_plt.save_figure(save_file_name, fig=fig, dpi=75)
    plt.close()

# #=========== Add coordinates ===============================
ax.annotate(r'($x_1, y_1$)', (x_coord_max+5, y_coord_max+5))
ax.annotate(r'($x_2, y_2$)', (rec_space_theor_shift_AlGaN[0][7]-50, rec_space_theor_shift_AlGaN[1][7]))
ax.annotate(r'($x_3, y_3$) = ($x_1, y_3+D(x_1-X_3)$)', (rec_space_theor_shift_AlGaN[0][7]+25, rec_space_theor_shift_AlGaN[1][7]-20))